In [9]:
# 

Annotation file reader

In [1]:
from pycocotools.coco import COCO

ann_file = "C:\\Users\\S_G\Desktop\\yolo_dataset\\coco\\annotations_trainval2017\\annotations\\instances_val2017.json"
# ann_file = "C:\\Users\\S_G\\Documents\\GitHub\\Shape_Detection\\3_shape_annotation.json"
coco=COCO(ann_file)
category_ids = coco.getCatIds(['bicycle'])


loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


In [2]:
image_ids = coco.getImgIds(catIds=[2])
print("id",image_ids[0:5])
annotation_ids = coco.getAnnIds(imgIds=169996, catIds=[2])
print(len(annotation_ids))

id [184324, 546823, 343561, 169996, 277005]
2


In [3]:
anns = coco.loadAnns(annotation_ids)
for ann in anns:
    print(ann['bbox'])

[151.74, 343.74, 108.39, 96.0]
[399.89, 329.15, 81.47, 98.39]


Augmentation

Data loader Pipeline

In [ ]:

from torch.utils.data import DataLoader
import config

def get_loaders(train_csv_path, test_csv_path):
    from dataset import YOLODataset

    IMAGE_SIZE = config.IMAGE_SIZE
    train_dataset = YOLODataset(
        train_csv_path,
        transform=config.train_transforms,
        S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        img_dir=config.IMG_DIR,
        label_dir=config.LABEL_DIR,
        anchors=config.ANCHORS,
    )
    test_dataset = YOLODataset(
        test_csv_path,
        transform=config.test_transforms,
        S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        img_dir=config.IMG_DIR,
        label_dir=config.LABEL_DIR,
        anchors=config.ANCHORS,
    )
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=True,
        drop_last=False,
    )
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=False,
        drop_last=False,
    )

    train_eval_dataset = YOLODataset(
        train_csv_path,
        transform=config.test_transforms,
        S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        img_dir=config.IMG_DIR,
        label_dir=config.LABEL_DIR,
        anchors=config.ANCHORS,
    )
    train_eval_loader = DataLoader(
        dataset=train_eval_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=False,
        drop_last=False,
    )

    return train_loader, test_loader, train_eval_loader

train_loader, test_loader, train_eval_loader = get_loaders(
        train_csv_path=config.DATASET + "/train.csv", test_csv_path=config.DATASET + "/test.csv"
    )

Read Json

In [9]:
import json

def get_class_encode(label):
    return label;
def get_in_yolo(cords):
    x1,y1,x2,y2 = cords[0]['x'],cords[0]['y'],cords[1]['x'],cords[1]['y']
    return x1, y1, (x2-x1), (y2-y1)
# file = open('3_shape_annotation.json')
# # returns JSON object as 
# # a dictionary
# data = json.load(file)
  
# # Iterating through the json
# # list
# count = 0
# for i in data['annotations']:
#     count += 1
#     class_id = get_class_encode(i['label'])
#     x,y,w,h = get_in_yolo(i['coordinates'])
#     print(f'class = {class_id}, x,y= {x,y} w,h = {w,h}')
# print("\nTotal = ", count)
# # Closing file
# file.close()
def json_to_yolo(input_file_name, output_file_name):
    # get coordinates, height and width from below lambda func.
    get_cords = lambda cords : [cords[0]['x'],cords[0]['y'],cords[1]['x'],cords[1]['y'], (cords[1]['x'] - cords[0]['x']) , (cords[1]['y'] - cords[0]['y'])]
    file = open(input_file_name)
    data = json.load(file)
    # Iterating through the json
    # list
    # print(data['metadata']['system']['width'])
    height , width = data['metadata']['system']['height'], data['metadata']['system']['width']
    count = 0
    FILE = f''
    for i in data['annotations']:
        count += 1
        class_id = get_class_encode(i['label'])
        x,y,x_2,y_2,w,h = get_cords(i['coordinates'])
        print(f'class = {class_id}, x,y= {x,y} w,h = {w,h}')
        x_c , y_c = (x+w/2)/ width , (y+h/2)/height
        FILE += f'{class_id[5:]} {x_c} {y_c} {w} {h}\n'
    print("\nTotal = ", count)
    # Closing file
    f = open(output_file_name, "w")
    f.write(FILE)
    f.close()

json_to_yolo('3_shape_annotation.json', '3_shape_annotation.txt')

class = Class0, x,y= (1244.9, 773.31) w,h = (25.24000000000001, 56.98000000000002)
class = Class0, x,y= (1282.4, 837.5) w,h = (22.3599999999999, 50.49000000000001)
class = Class0, x,y= (872.7848478357405, 927.8581943540748) w,h = (33.43513169236826, 47.982779698868626)
class = Class0, x,y= (909.5022518936287, 734.5593195911085) w,h = (45.96774810637157, 27.24190899482369)
class = Class0, x,y= (929.7796539561557, 896.9927478437248) w,h = (45.41518291930993, 41.7272521562752)
class = Class0, x,y= (889.9920796543322, 794.3105786016323) w,h = (49.42792034566776, 33.027312415144365)
class = Class0, x,y= (772.6513953790497, 918.7963939007611) w,h = (49.47296110089087, 32.145512478052524)
class = Class0, x,y= (841.31, 741.6301969997758) w,h = (54.559999999999945, 28.37980300022423)
class = Class0, x,y= (752.8398955634535, 784.22) w,h = (35.75010443654651, 44.96008556547281)
class = Class0, x,y= (810.5721541386026, 790.64) w,h = (31.631187922162553, 50.927347475126)
class = Class0, x,y= (954.5

Add custom layer to yolo

Training

Testing